# orienté objet (copie avec modif de raphael)

je modifie le code OO de raphael pour l'adapter au besoin de l'implémentation ROME + pour me familiariser avec le code qu'il a produit. Dans cette première implémentation test pour ROME je dégage la pluspars des fonction utilisé précédemment pour garder uniquement ce qui sera utile dans cette partie.

J'ai enlever le truc de batch et la mise sur GPU parce que j'y comprend R.

In [2]:
from transformers import GPT2TokenizerFast, GPT2LMHeadModel
import torch
from functools import partial
import torch.nn.functional as F

/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
#inputs = les prompts sur lesquels on va calculer k* et v*
#subject le sujet pour qui il faut determiner k*,v*
class Instance_for_ROME :
    def __init__(self, subject, inputs= None, l_star = 18, model_name = 'gpt2-xl', nb_prompt=50):
        self.model_name = model_name
        self.subject = subject
        self._l_star = l_star

        self.model = GPT2LMHeadModel.from_pretrained(model_name)
        self.tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
        self.tokenizer.pad_token = self.tokenizer.eos_token

        if inputs == None:
            self.generate_prompts(50)
        else:
            self.prompts = inputs

        self._subject_mask = self.compute_subject_mask()
        self._last_subject_indices= (self._subject_mask * torch.arange(1, self._subject_mask.shape[1] + 1, device=self._subject_mask.device)).argmax(dim=1)

        self._ks = None
        self._k_star=None
        self._hooks = []
        self._logits = None
        self.output = None

    def __str__(self):
        return f'Instance of {self.model.config.architectures[0]} model'
    
    def tokenize(self,batch,offsetsMapping=False):
        inputs=self.tokenizer(batch,return_tensors='pt',padding=True,return_offsets_mapping=offsetsMapping)
        return inputs
    
    def compute_subject_mask(self, prompts = None, subject = None):
        res =[]

        if prompts == None:
            prompts = self.prompts
        if subject == None:
            subject = self.subject

        input = self.tokenize(prompts,offsetsMapping=True)
        mask=[]
        for j, prompt in enumerate(prompts):
            map = torch.zeros_like(input.input_ids[j], dtype=torch.int)
            for i,t in enumerate(input.offset_mapping[j]):
                if (prompts[j].find(subject)-1<=t[0]) and (t[1]<=prompts[j].find(subject)+len(subject)) and (prompts[j].find(subject) !=-1):
                    map[i] = 1
            mask.append(map)
        subject_mask = torch.stack(mask)
        subject_mask = torch.logical_and(subject_mask, input.attention_mask).int()
        return subject_mask
    
    def get_ks_hook(self, last_subject_indices = None):
        if last_subject_indices == None:
            last_subject_indices = self._last_subject_indices
        
        def hook(module,input,output):
            if isinstance(output, torch.Tensor):
                res = output[torch.arange(len(last_subject_indices)), last_subject_indices]
                self._ks = res
            else:
                raise TypeError("Expected output to be a torch.Tensor, but got {}".format(type(output)))
            pass
        
        return hook
    
    def accroche(self, l_star = None):
        if l_star == None:
            l_star = self._l_star
        hook = self.get_ks_hook()
        handle = self.model.transformer.h[l_star].mlp.c_fc.register_forward_hook(hook)
        self._hooks.append(handle)
        pass
    
    def enleve(self):
        for handle in self._hooks:
            handle.remove()
        self._hooks = []
        pass
    
    def run(self, conserve_logits = False,conserve_output = False):
        input = self.tokenize(self.prompts)
        with torch.no_grad():
            output = self.model(**input, labels = input.input_ids) 
        if self._ks != None:
            self._k_star = torch.mean(self._ks, dim=0)
        if conserve_logits:
            self._logits = output.logits 
        if conserve_output:
            self._output = output
        pass

    def generate_prompts(self, nb_prompt, min_len = 2, max_len = 11):
        vocab_size = self.tokenizer.vocab_size
        nb_token = torch.randint(min_len, max_len, (nb_prompt,))
        max_tokens = nb_token.max() 
        tokens = torch.randint(0, vocab_size, (nb_prompt, max_tokens))
        padded_tokens = F.pad(tokens, (0, max_tokens - nb_token.max().item()), value=vocab_size)
        decoded_sequences = [self.tokenizer.decode(seq[:nb_token[i].item()]) for i, seq in enumerate(padded_tokens)]
        res = [x + ' ' + self.subject for x in decoded_sequences]
        self.__init__(self.subject, res, self._l_star,self.model_name)
        pass

    def get_k_star(self,l_star = None):
        self.accroche(l_star)
        self.run()
        self.enleve()
        return self._k_star

In [17]:
test = Instance_for_ROME('Eiffel Tower')

[' favorite UnlePresent Eiffel Tower', ' AustrianactivelysesLOD Brawl handguneman relativeANCE Eiffel Tower', ' vegetable Abraham Eiffel Tower', ' Root bouAdult820 Eiffel Tower', ' Nut aggressive 384 quickest Koran Setup heels Eiffel Tower', ' depressed dagger Saunders constitute summar Mercy substantially somisode Eiffel Tower', '260 pure anarchyanos confused pedestrian corners Empires crowned757 Eiffel Tower', ' SentConsuationgrain assumed Eiffel Tower', ' ================= 560 payingSourceFile 421 stressed Eiffel Tower', ' rulesama Samuel Eiffel Tower', ' Functionalč SE Mineral repository turnedsem GN bandwagonOHN Eiffel Tower', 'lt fielding Eiffel Tower', ' collideproc simmer Wat Eiffel Tower', ' blownpages Eiffel Tower', ' backs inadvertVT Sussex Theyaghanhouse463 Goodman Claire Eiffel Tower', ' sharingden ACEWA ambiguity newest Emergency ******************************** Eiffel Tower', ' chicken infinity332++; BBQ hack flickeringNonetheless Eiffel Tower', ' costing ra originated s

In [5]:
test.generate_prompts(50)
test.get_k_star()

tensor([-2.1392, -1.6501, -0.7724,  ..., -1.3727, -1.3840, -2.6023])

# test

In [4]:
import requests
from transformers import GPT2TokenizerFast, GPT2LMHeadModel
import torch.nn.functional as F
import torch

In [7]:
prompts = [ "Roses are red",
            "violets are blue",
            "once upon a time"]

In [8]:
class test:
    def __init__(self, prompts, model_name = 'gpt2'):
        self.model = GPT2LMHeadModel.from_pretrained(model_name)
        self.tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.nb_prompt = len(prompts)
        self.inputs = self.tokenizer(prompts, return_tensors='pt',padding=True)
        self.nb_token = len(self.inputs.input_ids[0])
        self.observe = None
        self.hooks = []
    def run(self):
        self.model( **self.inputs, labels = self.inputs.input_ids)
        pass
    def accroche(self,i):
        handle = self.model.transformer.h[i].mlp.c_fc.register_forward_hook(self.def_hook())
        self.hooks.append(handle)
        pass
    def def_hook(self):
        def hook(module,input,output):
            self.observe = output
            pass
        return hook
    def decroche(self):
        while len(self.hooks)!=0:
            handle = self.hooks.pop()
            handle.remove
        pass


In [9]:
test = test(prompts,'gpt2-xl')

In [10]:
test.model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1600)
    (wpe): Embedding(1024, 1600)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-47): 48 x GPT2Block(
        (ln_1): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=4800, nx=1600)
          (c_proj): Conv1D(nf=1600, nx=1600)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=6400, nx=1600)
          (c_proj): Conv1D(nf=1600, nx=6400)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1600, out_features=50257, bias=False)
)

In [11]:
test.accroche(5)

In [12]:
test.run()

In [13]:
test.observe

tensor([[[-0.1719, -1.7652, -1.2129,  ..., -1.9756, -0.2020,  0.0178],
         [-0.4384, -2.8245, -1.5303,  ..., -1.0836, -0.9600,  0.0858],
         [-0.1192, -1.4361, -1.8975,  ..., -1.6589, -0.1612, -0.0735],
         [ 0.0581, -3.9133, -1.7499,  ..., -2.6801, -1.0978, -0.1239],
         [-0.0636, -1.3829, -1.7825,  ..., -1.7437, -1.0203, -0.0888]],

        [[-0.1709, -1.7098, -1.1889,  ..., -2.1661,  0.3088, -0.3459],
         [-0.0392, -3.3459, -0.1746,  ..., -2.6891, -0.7095,  0.0816],
         [-0.3265, -2.9280, -2.9074,  ..., -1.6303, -0.6460, -0.1716],
         [-0.0211, -1.2382, -1.3937,  ..., -1.7412, -0.1739, -0.1458],
         [-0.2472, -2.5032,  0.3609,  ..., -1.5566, -0.1479, -0.6963]],

        [[-0.2837, -3.2428, -2.1223,  ..., -0.6043,  0.3078,  0.4584],
         [-0.1778, -2.6204, -0.7883,  ..., -1.9352, -0.7247, -0.2274],
         [ 0.1209, -1.5343, -2.1487,  ..., -2.4988,  0.0497, -0.1618],
         [-0.3154, -4.7122, -2.3165,  ..., -3.5406, -0.3736, -0.1380],
  

## Compute v*

Je recopie ici le code de la classe histoire de faire mes propres tests et de pas toucher au code fait avant moi

Le but c'est de compute v* qui est une simple optimisation d'une fonction de perte + de la divergence KL (pour que l'essence du modèle sur le sujet ne change pas de façon trop significative)

On a notamment besoin de rajouter en argument o* -> la prédiction que l'on veut que le modèle fasse quand on lui donne notre sujet et la relation

De même on a besoin de p, le prompt factuel qui donne clairement la relation entre s et o*
Typiquement: 'The Space Needle is in Seattle"
On aura aussi besoin de p' qui permet de voir si notre modèle dévie pas trop du sens initial mais ça j'avoue pour l'instant c'est pas hyper clair.

In [5]:
from torch.optim import Adam

In [27]:
class ValueEditor :
    def __init__(self, instance, o_star):
        self.instance = instance
        self.o_star = o_star
        self._v_star = torch.nn.Parameter(torch.randn(instance._k_star.shape))

        self.hook_handle = None

    def mlp_output_hook(self, module, input, output):
        # output est la sortie MLP originale, on la remplace par v*
        return self._v_star.unsqueeze(0).expand_as(output)
    
    def accroche(self):
        # Hook sur la sortie du MLP à la même couche L*
        l_star = self.instance._l_star
        handle = self.instance.model.transformer.h[l_star].mlp.c_proj.register_forward_hook(self.mlp_output_hook)
        self._hook_handle = handle
    
    def enleve(self):
        if self._hook_handle is not None:
            self._hook_handle.remove()

In [29]:
def optimize_v_star(editor, prompt_templates, object_token_str, n_iter=100, lr=1e-2):
    instance = editor.instance
    tokenizer = instance.tokenizer
    model = instance.model

    editor.accroche()
    
    optimizer = torch.optim.Adam([editor._v_star], lr=lr)
    loss_fn = torch.nn.CrossEntropyLoss()

    input_prompts = [template.format(subject=instance.subject) for template in prompt_templates]
    tokenized = tokenizer(input_prompts, return_tensors="pt", padding=True)

    # Token ID cible à prédire
    target_token_id = tokenizer.encode(object_token_str, add_special_tokens=False)[0]

    for i in range(n_iter):
        optimizer.zero_grad()
        
        outputs = model(**tokenized)
        logits = outputs.logits  # [batch_size, seq_len, vocab_size]

        # Prédiction sur le dernier token
        last_token_logits = logits[:, -1, :]  # shape: [batch, vocab_size]
        
        targets = torch.tensor([target_token_id] * last_token_logits.shape[0])
        loss = loss_fn(last_token_logits, targets)

        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"[{i}] loss = {loss.item():.4f}")

    editor.enleve()
    return editor._v_star.detach()

In [30]:
instance = Instance_for_ROME(subject)
instance.get_k_star()

tensor([-2.4697, -1.0214, -0.2980,  ..., -0.0869, -0.6566,  0.3382])

In [31]:
subject = 'Paris'
o_star = 'Italy'
factual_prompts = [
    '{subject} is the capital of'
    'In which country is {subject} located?'
    'Which country has {subject} as its capital'
    'What country is home to {subject}?'

]

editor = ValueEditor(instance, o_star)

v_star = optimize_v_star(editor, factual_prompts, o_star)

print(v_star)


RuntimeError: The expanded size of the tensor (1600) must match the existing size (6400) at non-singleton dimension 2.  Target sizes: [1, 26, 1600].  Tensor sizes: [1, 6400]